In [1]:
import rasterio as rio
from rasterio.features import rasterize
from rasterio.windows import Window
import numpy as np
import geopandas as gpd
from shapely.geometry import box
import sys,os
import concurrent.futures
from rasterio import windows
from ezflow.models import build_model
from ezflow.models import get_default_model_cfg
from matplotlib import pyplot as plt
import torch 


In [2]:
def rasterize_from_profile(geometry_iter, c_profile, burn_value):
    """
    rasterize shapes of geometry iterator using the background profile and the burn value.
    returns a numpy array of raster
    """
    out_raster = rasterize(geometry_iter,
                     (c_profile["height"], c_profile["width"]),
                     fill=0,
                     transform=c_profile["transform"],
                     all_touched=True,
                     default_value=burn_value,
                     dtype=c_profile["dtype"])
    return out_raster

def rasterize_gdf(gdf, input_profile, burn_column="b_val"):
    """
    Inputs:
        gdf: geodataframe of geometries with burn value at column burn_column
        rio_dataset: rasterio dataset of refrence (background) image
        burn_column: column name in shapefile representing burning values
    """
            
    new_profile = input_profile
            
    rasterization_images=[]
    
    if (not (burn_column in gdf.columns)):
        single_rasterization= rasterize_from_profile(gdf.geometry, new_profile, 1)
        rasterization_images.append(single_rasterization)
        
    else:
        burning_values=np.sort(gdf[burn_column].unique())
        for c_burn_value in burning_values:
            c_burn_gdf_view = gdf[gdf[burn_column]==c_burn_value]
            single_rasterization = rasterize_from_profile(c_burn_gdf_view.geometry, new_profile, c_burn_value)
            rasterization_images.append(single_rasterization)
    return np.max(rasterization_images, axis=0)

def rasterize_view(args):
    """
    Runs rasterization on a windowed view
    """
    geometry_view, shapes_gdf, profile, burn_column = args
    view_gdf = gpd.GeoDataFrame(geometry=[geometry_view], crs=shapes_gdf.crs)
    gdf_view = gpd.overlay(shapes_gdf, view_gdf, "intersection")
    if (len(gdf_view)>0):
        return rasterize_gdf(gdf_view, profile, burn_column)
    else:
        return np.zeros((profile["height"], profile["width"]), dtype=profile["dtype"])

def parralel_rasterization(shapes_gdf, output_dst, burn_column):
    """
    Split rasterization process
    Inputs:
        shapes_gdf: geodataframe of all features
        output_dst: output rio dataset to fill
        burn_column: str defining burn column in shapes_gdf
    """
    
    windows_list = [window for ij, window in output_dst.block_windows()]
    window_geometries = [ box(*windows.bounds(c_window,output_dst.transform)) for c_window in windows_list]
            
    # windows.transform(c_window,output_dst.transform)
    concurrent_args = []
    for c_window, c_window_geometrie in zip(windows_list, window_geometries):
        c_profile = output_dst.profile.copy()
        c_profile.update( 
            height=c_window.height,
            width=c_window.width,
            transform = windows.transform(c_window,output_dst.transform)
            )
        concurrent_args.append( (c_window_geometrie, shapes_gdf, c_profile, burn_column) )
    
    with concurrent.futures.ProcessPoolExecutor(
            max_workers=8
        ) as executor:
        futures = executor.map(rasterize_view, concurrent_args)
        for window, result in zip(windows_list, futures):
            output_dst.write(result,1, window=window)
        

### Input data def

In [3]:
input_shapefile_1 = "C:/DATA_SANDBOX/Alignment2/MOURMELON/OSM_data/MOURMELON_osm_polys_polygons.shp"
input_shapefile_2 = "C:/DATA_SANDBOX/Alignment2/MOURMELON/17MAY17110802/17MAY17110802_bativec.shp"

In [4]:
# Load shapefile
in_gdf_1 = gpd.read_file(input_shapefile_1)
in_gdf_2 = gpd.read_file(input_shapefile_2)
assert in_gdf_1.crs == in_gdf_2.crs, "Input shapefiles must share the same crs"

In [5]:
# compute common extents
gdf_1_box = box(*in_gdf_1.total_bounds)
gdf_2_box = box(*in_gdf_2.total_bounds)

common_extent = (gdf_1_box.union(gdf_2_box)).bounds

In [6]:
def extents_to_profile(extent, gsd=0.5, **kwargs):
    
    in_height = round((extent[2]-extent[0])/gsd)
    in_width = round((extent[3]-extent[1])/gsd)
    in_transform = rio.transform.from_origin(extent[0], extent[-1], gsd,gsd)
    rasterization_profile = {
    "driver": "GTiff", "count":1, "height": in_height, "width":in_width,
    "dtype":rio.uint8, "transform":in_transform
    }
    rasterization_profile.update(kwargs)
    return rasterization_profile

In [7]:
rasterized_profile = extents_to_profile(common_extent, crs=in_gdf_1.crs)

In [ ]:
rasterized_shp1 = rasterize_from_profile(in_gdf_1.geometry, rasterized_profile, 1)
rasterized_shp2 = rasterize_from_profile(in_gdf_2.geometry, rasterized_profile, 1)

In [ ]:
offset_x=1200
offset_y=1200
patch_size=256
s1 = rasterized_shp1[offset_x:offset_x+patch_size, offset_y:offset_y+patch_size]
s2 = rasterized_shp2[offset_x:offset_x+patch_size, offset_y:offset_y+patch_size]

s1 = torch.Tensor(s1)
s2 = torch.Tensor(s2)

s1=s1.repeat(3,1,1)
s2=s2.repeat(3,1,1)

#s1 = s1.permute(1,2,0)
#s2 = s2.permute(1,2,0)

In [ ]:
raft_cfg = get_default_model_cfg("RAFT")
raft_cfg.ENCODER.CORR_RADIUS = 5
model = build_model("RAFT", cfg=raft_cfg)

In [ ]:
model = build_model("FlowNetC", default=True)

In [ ]:
model.forward(s1,s2)

In [ ]:
s1_p= "C:/Users/cherif/Downloads/Sampler.tar/Sampler/Driving/RGB_cleanpass/left/0400.png"
s2_p= "C:/Users/cherif/Downloads/Sampler.tar/Sampler/Driving/RGB_cleanpass/right/0400.png"

In [ ]:
import ezflow
ezflow.__version__

In [ ]:
from ezflow.models import Predictor
from torchvision.transforms import Resize

predictor = Predictor("RAFT", default=True,
    data_transform=Resize((256, 256))
)
flow = predictor(s1_p, s2_p)